In [121]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from xgboost import XGBClassifier

# JSON: para salvar hiperparâmetros como texto no CSV final
import json

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report, f1_score, average_precision_score)
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import accuracy_score


# ParameterSampler para sorteio de combinações aleatórias de hiperparâmetros
# TimeSeriesSplit é para validação cruzada respeitando ordem temporal
from sklearn.model_selection import ParameterSampler, TimeSeriesSplit

In [122]:
def preparar_dataset(df, col_data="data", col_cidade="localidade"):
    """
    Função:
    Faz uma cópia do DataFrame para não alterar o original.
    Converte a coluna de data para datetime.
    Remove linhas onde a data ficou inválida (NaT).
    Ordena por cidade e data para manter consistência temporal.
    Retorna o DataFrame pronto.
    """

    # Cria uma cópia do DataFrame original
    df_local = df.copy()

    # Converte a coluna de data para datetime; erros viram NaT
    df_local[col_data] = pd.to_datetime(df_local[col_data], errors="coerce")

    # Remove linhas que não têm data válida
    df_local = df_local.dropna(subset=[col_data])

    # Ordena por cidade e data para garantir ordem temporal
    df_local = df_local.sort_values([col_cidade, col_data]).reset_index(drop=True)

    # Retorna DataFrame pronto
    return df_local

In [123]:
def montar_X_y_com_onehot(df_cidade, feature_cols, col_target):
    """
    Função:
    Separa X (features) e y (target).
    Identifica colunas object (texto).
    Converte essas colunas para one-hot via get_dummies.
    Retorna X (já numérico) e y (int).
    """

    # X recebe apenas as colunas de features
    X = df_cidade[feature_cols].copy()

    # y recebe a coluna target e garante tipo int (0/1)
    y = df_cidade[col_target].astype(int).copy()

    # Identifica colunas de texto (dtype object)
    cols_obj = X.select_dtypes(include=["object"]).columns.tolist()

    # Se existirem colunas texto, faz one-hot nelas
    # dummy_na=True cria uma categoria para valores nulos (NaN)
    if len(cols_obj) > 0:
        X = pd.get_dummies(
            X,
            columns=cols_obj,
            dummy_na=True,
            drop_first=False
        )
        
        # Retorna X e y
    return X, y

In [124]:
def split_temporal(X, y, proporcao_treino=0.8):
    """
    Função:
    Calcula um índice de corte baseado na proporção.
    Separa treino e teste respeitando a ordem temporal.
    Retorna X_train, y_train, X_test, y_test.
    """

    # Total de linhas
    n_total = len(X)

    # Índice de corte ***por exemplo 80% do total
    idx_corte = int(np.floor(proporcao_treino * n_total))

    # Treino recebe as primeiras linhas
    X_train = X.iloc[:idx_corte].reset_index(drop=True)
    y_train = y.iloc[:idx_corte].reset_index(drop=True)

    # Teste recebe as últimas linhas
    X_test = X.iloc[idx_corte:].reset_index(drop=True)
    y_test = y.iloc[idx_corte:].reset_index(drop=True)

    # Retorna conjuntos
    return X_train, y_train, X_test, y_test

In [125]:
def alinhar_colunas_treino_teste(X_train, X_test):
    """
    Esta função garante que:
    - X_train e X_test tenham exatamente as mesmas colunas.
    - Se uma coluna existir em um e não no outro, ela é criada com valor 0.
    """

    # align(..., join="left") garante que X_test tenha pelo menos as colunas do treino
    # fill_value=0 preenche colunas ausentes com zeros
    X_train_al, X_test_al = X_train.align(X_test, join="left", axis=1, fill_value=0)

    # Retorna alinhados
    return X_train_al, X_test_al


In [126]:
def calcular_scale_pos_weight(y):
    """
    scale_pos_weight = (n_negativos / n_positivos)
    Isso ajuda o XGBoost a não ignorar a classe minoritária.
    """

    # Conta negativos e positivos
    neg = int((y == 0).sum())
    pos = int((y == 1).sum())

    # Evita divisão por zero
    if pos == 0:
        return 1.0

    # Retorna a razão
    return float(neg / pos)

In [127]:
def calcular_metricas_classificacao(y_true, y_proba, threshold=0.5):
    y_pred = (y_proba >= threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    
    accuracy = accuracy_score(y_true, y_pred)

    if len(np.unique(y_true)) == 2:
        roc_auc = roc_auc_score(y_true, y_proba)
        pr_auc = average_precision_score(y_true, y_proba)
    else:
        roc_auc = np.nan
        pr_auc = np.nan

    return {
        "threshold": float(threshold),
        "tp": int(tp),
        "tn": int(tn),
        "fp": int(fp),
        "fn": int(fn),
        "accuracy": float(accuracy),   # <<< CHAVE QUE ESTAVA FALTANDO
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "roc_auc": float(roc_auc) if roc_auc == roc_auc else np.nan,
        "pr_auc": float(pr_auc) if pr_auc == pr_auc else np.nan
    }


In [128]:
"""
def escolher_threshold_no_treino(y_true, y_proba, metrica="f1"):
    
    Esta função tenta vários thresholds e escolhe o melhor baseado em:
    - f1
    - recall
    - precision

    Ela testa thresholds de 0.10 até 0.90, passo 0.05
    

    # Cria lista de thresholds
    thresholds = np.round(np.arange(0.10, 0.91, 0.05), 2)

    # Define melhor threshold e melhor valor
    melhor_thr = 0.5
    melhor_val = -np.inf

    # Testa cada threshold
    for thr in thresholds:
        # Calcula métricas com esse threshold
        m = calcular_metricas_classificacao(y_true, y_proba, threshold=thr)

        # Pega a métrica escolhida
        val = m.get(metrica, np.nan)

        # Atualiza se for melhor
        if val == val and val > melhor_val:
            melhor_val = val
            melhor_thr = thr

    # Retorna melhor threshold encontrado
    return float(melhor_thr)
"""

def escolher_threshold_no_treino(y_true, y_proba, metrica="f1", alpha_recall=0.5):
    """
    Esta função tenta vários thresholds e escolhe o melhor baseado em uma métrica.
    Testa thresholds de 0.10 até 0.90 a cada 0.05 passos
    Métricas:
    - "f1"
    - "recall"
    - "precision"
    - "accuracy"
    - "recall_accuracy" -> score composto:
          score = alpha_recall*recall + (1-alpha_recall)*accuracy
    """

    thresholds = np.round(np.arange(0.10, 0.91, 0.05), 2)

    melhor_thr = 0.5
    melhor_val = -np.inf

    for thr in thresholds:
        m = calcular_metricas_classificacao(y_true, y_proba, threshold=thr)

        # Se a métrica for composta calcula o result
        if metrica == "recall_accuracy":
            val = (alpha_recall * m["recall"]) + ((1.0 - alpha_recall) * m["accuracy"])
        else:
            val = m.get(metrica, np.nan)

        if val == val and val > melhor_val:
            melhor_val = val
            melhor_thr = thr

    return float(melhor_thr)


In [129]:
def avaliar_params_com_validacao_temporal(
    X_train,
    y_train,
    params,
    n_splits=4,
    scoring="recall_accuracy",
    alpha_recall=0.5,
    random_state=42
):
    """
    scoring="recall_accuracy" => score = alpha_recall*recall + (1-alpha_recall)*accuracy
    - threshold 0.5 na validação para comparar hiperparâmetros de forma consistente.
    """

    tss = TimeSeriesSplit(n_splits=n_splits)
    scores = []

    for idx_tr, idx_val in tss.split(X_train):
        X_tr = X_train.iloc[idx_tr]
        y_tr = y_train.iloc[idx_tr]

        X_val = X_train.iloc[idx_val]
        y_val = y_train.iloc[idx_val]

        if len(np.unique(y_tr)) < 2 or len(np.unique(y_val)) < 2:
            continue

        spw = calcular_scale_pos_weight(y_tr)

        model = XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=random_state,
            n_jobs=-1,
            scale_pos_weight=spw,
            **params
        )

        model.fit(X_tr, y_tr)

        y_proba_val = model.predict_proba(X_val)[:, 1]
        y_pred_val = (y_proba_val >= 0.5).astype(int)

        # Métricas base para o score
        rec = recall_score(y_val, y_pred_val, zero_division=0)
        acc = accuracy_score(y_val, y_pred_val)

        if scoring == "recall_accuracy":
            score = (alpha_recall * rec) + ((1.0 - alpha_recall) * acc)
        elif scoring == "recall":
            score = rec
        elif scoring == "accuracy":
            score = acc
        elif scoring == "pr_auc":
            score = average_precision_score(y_val, y_proba_val)
        elif scoring == "roc_auc":
            score = roc_auc_score(y_val, y_proba_val)
        else:
            score = (alpha_recall * rec) + ((1.0 - alpha_recall) * acc)

        scores.append(score)

    if len(scores) == 0:
        return -np.inf

    return float(np.mean(scores))


In [ ]:
def treinar_xgb_por_cidade_com_tuning(
    df,
    col_cidade="localidade",
    col_data="data",
    col_target="chove_amanha_vtr",
    proporcao_treino=0.8,
    min_linhas_cidade=500,
    n_iter=50,
    n_splits_tss=4,
    scoring_valid="pr_auc",
    usar_threshold_otimo=True,
    metrica_threshold="f1",
    random_state=42
):
    """
    Função objetivo
    - prepara dataset 
    - itera por cidade
    - monta X e y com one-hot
    - split temporal
    - tuning com random search
    - treina modelo final com melhores params
    - escolhe threshold no treino (opcional)
    - avalia no teste
    - salva 1 linha de resultados/métricas por cidade
    """

    # Prepara dataset ****data datetime e ordenação
    df_local = preparar_dataset(df, col_data=col_data, col_cidade=col_cidade)

    # Define colunas que não são features
    colunas_nao_features = {col_cidade, col_data, col_target}

    # Define lista de features como todas as colunas exceto as não-features
    feature_cols = [c for c in df_local.columns if c not in colunas_nao_features]

    # Define espaço de hiperparâmetros 
    param_distributions = {
        "n_estimators": [200, 400, 600, 800],
        "learning_rate": [0.01, 0.03, 0.05, 0.1],
        "max_depth": [2, 3, 4, 5, 6],
        "subsample": [0.7, 0.8, 0.9, 1.0],
        "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
        "min_child_weight": [1, 3, 5, 7, 10],
        "reg_lambda": [1.0, 2.0, 5.0, 10.0],
        "gamma": [0, 0.5, 1.0, 2.0]
    }

    # Gera combinações aleatórias de hiperparâmetros
    sampler = ParameterSampler(
        param_distributions=param_distributions,
        n_iter=n_iter,
        random_state=random_state
    )
    
    #scoring_valid="recall"
    #metrica_threshold="recall"

    # Lista para guardar resultados finais
    resultados = []

    # Lista de cidades únicas
    cidades = df_local[col_cidade].dropna().unique()

    # Itera por cidade
    for cidade in cidades:
        # Filtra dados da cidade
        df_cid = df_local[df_local[col_cidade] == cidade].copy()

        # Conta total de linhas
        n_total = int(len(df_cid))

        # Se tiver poucas linhas, marca e continua
        if n_total < min_linhas_cidade:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": f"n_total < {min_linhas_cidade}",
                "n_total": n_total
            })
            continue

        # Ordena por data dentro da cidade
        df_cid = df_cid.sort_values(col_data).reset_index(drop=True)

        # Monta X e y já com one-hot
        X, y = montar_X_y_com_onehot(df_cid, feature_cols=feature_cols, col_target=col_target)

        # Se total tiver apenas uma classe, não dá para treinar
        if len(np.unique(y)) < 2:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "apenas_uma_classe_no_total",
                "n_total": n_total
            })
            continue

        # Split temporal
        X_train, y_train, X_test, y_test = split_temporal(X, y, proporcao_treino=proporcao_treino)

        # Alinha colunas de treino e teste 
        X_train, X_test = alinhar_colunas_treino_teste(X_train, X_test)

        # Se treino ou teste tiver apenas uma classe, não dá para avaliar
        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "treino_ou_teste_sem_duas_classes",
                "n_total": n_total,
                "n_treino": int(len(y_train)),
                "n_teste": int(len(y_test))
            })
            continue

        # Variáveis para melhor resultado
        melhor_score = -np.inf
        melhor_params = None

        # "Tuning" para testar combinações de hiperparâmetros
        for trial_id, params in enumerate(sampler):
            score = avaliar_params_com_validacao_temporal(
                X_train=X_train,
                y_train=y_train,
                params=params,
                n_splits=n_splits_tss,
                scoring="recall_accuracy",
                alpha_recall=0.5,   # 50% recall, 50% accuracy
                random_state=random_state
            )

            # Atualiza se for melhor
            if score > melhor_score:
                melhor_score = score
                melhor_params = params

        # Se não achou trial válido
        if melhor_params is None or melhor_score == -np.inf:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "nenhum_trial_valido",
                "n_total": n_total
            })
            continue

        # Calcula scale_pos_weight no treino inteiro
        spw_final = calcular_scale_pos_weight(y_train)

        # Cria modelo final com melhores parâmetros
        model_final = XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=random_state,
            n_jobs=-1,
            scale_pos_weight=spw_final,
            **melhor_params
        )

        # Treina modelo final
        model_final.fit(X_train, y_train)

        # Probabilidade no treino ****para escolher threshold
        y_proba_train = model_final.predict_proba(X_train)[:, 1]

        # Probabilidade no teste *****para métricas finais
        y_proba_test = model_final.predict_proba(X_test)[:, 1]

        # Escolhe threshold no treino 
        if usar_threshold_otimo:
            thr_escolhido = escolher_threshold_no_treino(
                y_true=y_train,
                y_proba=y_proba_train,
                metrica=metrica_threshold,
                alpha_recall=0.5
            )
        else:
            thr_escolhido = 0.5

        # Calcula métricas no teste
        metricas_teste = calcular_metricas_classificacao(
            y_true=y_test,
            y_proba=y_proba_test,
            threshold=thr_escolhido
        )

        # Informações temporais: ano início e ano fim
        ano_inicio = int(df_cid[col_data].dt.year.min())
        ano_fim = int(df_cid[col_data].dt.year.max())

        # Prevalência de chuva no total e no teste
        preval_total = float(y.mean())
        preval_teste = float(y_test.mean())

        # Tamanho do teste
        n_teste = int(len(y_test))

        # Nível de confiabilidade baseado no tamanho do teste
        if n_teste < 200:
            confiabilidade = "baixa"
        elif n_teste < 800:
            confiabilidade = "media"
        else:
            confiabilidade = "alta"

        # Número final de features (após one-hot)
        n_features = int(X_train.shape[1])

        # Salva resultado final da cidade
        resultados.append({
            "cidade": cidade,
            "cidade_insuficiente": False,
            "motivo": "",
            "ano_inicio": ano_inicio,
            "ano_fim": ano_fim,
            "n_total": n_total,
            "n_treino": int(len(y_train)),
            "n_teste": n_teste,
            "n_features": n_features,
            "prevalencia_total": preval_total,
            "prevalencia_teste": preval_teste,
            "confiabilidade": confiabilidade,
            "scoring_valid": scoring_valid,
            "best_valid_score": float(melhor_score),
            "best_params_json": json.dumps(melhor_params, ensure_ascii=False),
            "score_valid": float(melhor_score),
            "score_teste": float((0.5 * metricas_teste["recall"]) + (0.5 * metricas_teste["accuracy"])),
            "alpha_recall": 0.5,
            **metricas_teste
        })

    # Converte lista de resultados em DataFrame
    df_resultados = pd.DataFrame(resultados)

    # Retorna DataFrame final
    return df_resultados

In [131]:
if __name__ == "__main__":
    # Para ler o caminho do excel
    caminho_excel = r"C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v9.xlsx"

    
    # Para ler o excel
    df = pd.read_excel(caminho_excel)

    # Rodar estrutura de treinamento
    df_metricas_cidades = treinar_xgb_por_cidade_com_tuning(
        df=df,
        col_cidade="localidade",
        col_data="data",
        col_target="chove_amanha_vtr",
        proporcao_treino=0.8,
        min_linhas_cidade=500,
        n_iter=100,               
        n_splits_tss=4,
        scoring_valid="recall_accuracy",
        usar_threshold_otimo=True,
        metrica_threshold="recall_accuracy",
        random_state=42
    )
    print(df_metricas_cidades)

          cidade  cidade_insuficiente motivo  ano_inicio  ano_fim  n_total  \
0   AliceSprings                False               2008     2016     2224   
1       Brisbane                False               2008     2017     2979   
2         Cairns                False               2008     2016     2444   
3       Canberra                False               2007     2012     1092   
4          Cobar                False               2009     2010      534   
5   CoffsHarbour                False               2009     2014     1383   
6         Darwin                False               2008     2017     3063   
7         Hobart                False               2008     2017     1944   
8      Melbourne                False               2008     2017     4832   
9        Mildura                False               2009     2017     2595   
10         Moree                False               2009     2016     1915   
11  MountGambier                False               2008     201

In [132]:
#print(df_metricas_cidades)

In [133]:
#saida_csv = r"C:\Users\JacyzinGuilherme(Bip)\mentoria-bip\dados_editados\metricas_por_cidade_xgb_v3.csv"

df_metricas_cidades.to_csv(r"C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\metricas_por_cidade_xgb_v2.csv", sep=";", index=False)